<a href="https://colab.research.google.com/github/KaustubhSN12/Implement-Machine-Learning-with-Spark-or-Hadoop_BDA_LR/blob/main/Implement_Machine_Learning_with_Spark_or_Hadoop_BDA_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PySpark Linear Regression

In [2]:
#!pip install pyspark

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("linear_regression_model").getOrCreate()

In [5]:
real_estate = spark.read.option("inferSchema", "true").csv("/content/drive/MyDrive/Colab Notebooks/BDA_Practical_sem3/Pyspark_Linear_Regression-main/Real estate.csv",header=True)

In [6]:
real_estate.printSchema()

root
 |-- No: integer (nullable = true)
 |-- X1 transaction date: double (nullable = true)
 |-- X2 house age: double (nullable = true)
 |-- X3 distance to the nearest MRT station: double (nullable = true)
 |-- X4 number of convenience stores: integer (nullable = true)
 |-- X5 latitude: double (nullable = true)
 |-- X6 longitude: double (nullable = true)
 |-- Y house price of unit area: double (nullable = true)



In [7]:
real_estate.show(2)

+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
| No|X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
|  1|           2012.917|        32.0|                              84.87882|                             10|   24.98298|   121.54024|                      37.9|
|  2|           2012.917|        19.5|                              306.5947|                              9|   24.98034|   121.53951|                      42.2|
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+
only showing top 2 rows



In [8]:
real_estate.describe().show()

+-------+-----------------+-------------------+------------------+--------------------------------------+-------------------------------+--------------------+--------------------+--------------------------+
|summary|               No|X1 transaction date|      X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|         X5 latitude|        X6 longitude|Y house price of unit area|
+-------+-----------------+-------------------+------------------+--------------------------------------+-------------------------------+--------------------+--------------------+--------------------------+
|  count|              414|                414|               414|                                   414|                            414|                 414|                 414|                       414|
|   mean|            207.5| 2013.1489710144933| 17.71256038647343|                    1083.8856889130436|              4.094202898550725|  24.969030072463745|  121.53336108

In [9]:
# VectorAssembler to transform data into feature columns

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=[
 'X1 transaction date',
 'X2 house age',
 'X3 distance to the nearest MRT station',
 'X4 number of convenience stores',
 'X5 latitude',
 'X6 longitude'],
 outputCol='features')

In [10]:
data_set = assembler.transform(real_estate)
data_set.select(['features','Y house price of unit area']).show(2)

+--------------------+--------------------------+
|            features|Y house price of unit area|
+--------------------+--------------------------+
|[2012.917,32.0,84...|                      37.9|
|[2012.917,19.5,30...|                      42.2|
+--------------------+--------------------------+
only showing top 2 rows



In [11]:
# Split into Train and Test set
train_data,test_data = data_set.randomSplit([0.7,0.3])

In [12]:
train_data.show(truncate=False)

+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+------------------------------------------------+
|No |X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|features                                        |
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+------------------------------------------------+
|1  |2012.917           |32.0        |84.87882                              |10                             |24.98298   |121.54024   |37.9                      |[2012.917,32.0,84.87882,10.0,24.98298,121.54024]|
|2  |2012.917           |19.5        |306.5947                              |9                              |24.98034   |121.53951   |42.2                  

In [13]:
test_data.show(truncate=False)

+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+-----------------------------------------------+
|No |X1 transaction date|X2 house age|X3 distance to the nearest MRT station|X4 number of convenience stores|X5 latitude|X6 longitude|Y house price of unit area|features                                       |
+---+-------------------+------------+--------------------------------------+-------------------------------+-----------+------------+--------------------------+-----------------------------------------------+
|6  |2012.667           |7.1         |2175.03                               |3                              |24.96305   |121.51254   |32.1                      |[2012.667,7.1,2175.03,3.0,24.96305,121.51254]  |
|9  |2013.5             |31.7        |5512.038                              |1                              |24.95095   |121.48458   |18.8                      

In [14]:
#Train your model (Fit your model with train data)

from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol='Y house price of unit area')
lrModel = lr.fit(train_data)

In [15]:
# Perform descriptive analysis with correlation

test_stats = lrModel.evaluate(test_data)
print(f"RMSE: {test_stats.rootMeanSquaredError}")
print(f"R2: {test_stats.r2}")
print(f"R2: {test_stats.meanSquaredError}")

RMSE: 9.187496158547857
R2: 0.5569720807861372
R2: 84.41008566333163
